<details>
<summary>Table of Contents</summary>

- [1. Standard Scaler Informer ](#1-standard-scaler-informer)
- [2. Standard Scaler PatchTST](#2-standard-scaler-patchtst)
- [3. MinMax (0, 1) Scaler Informer](#3-minmax-scaler-0-1-informer)
- [4. MinMax (0, 1) Scaler PatchTST](#4-minmax-scaler-0-1-patchtst)
- [5. MinMax (0, 5) Scaler Informer](#5-minmax-scaler-0-5-informer)
- [6. MinMax (0, 5) Scaler PatchTST](#6-minmax-scaler-0-5-patchtst)

</details>


Here we perform a check on DE dataset to confirm choice of loss function and scaler for our data.

This script is to run the models. Final results are in the notebook "Comparison". 

Please note, the cell content is almost identical. However, when duplicating code and changing some arguments, it becomes easier to store and read results (especially if you want to experiment with 1 subpart) and split long running time into subprocesses. 

In [1]:
import os
import pandas as pd
import subprocess
import shutil
from utils.helper import extract_metrics_from_output, convert_results_into_df

# 1. Standard Scaler Informer

In [2]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

# Arguments that will be used also for file names
model = "Informer"
dataset = 'DE_data.csv'
losses = ["MSE", "RMSE", "MAE"]
country = dataset[:2]

log_dir = f"logs/loss_choice/standard"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

log_file_path = f"{log_dir}/{model}_{country}.log"

In [ ]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "96"
lr = "0.0001"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Lists to store the results
informer_results_scaled, informer_results_unscaled = [], []

# Log file
with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to .ipynb output cell

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Arguments for the command
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 5 \
              --dec_in 5 \
              --c_out 5 \
              --des 'Exp' \
              --train_epochs 10 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --loss_fnc "{loss}" \
              --scaler_type standard \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()
            
            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(informer_results_scaled, scaled_metrics), (informer_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_96_24_loss_choice_for_DE', model='Informer', data='custom', root_path='/vol/cs-hu/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, seq_len=96, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=10, batch_size=32, patience=3, learning_rate=0.0001, des='Exp', lradj='type3', pct_start=0.3, us

In [ ]:
path_dir = './dataset_results'
csv_name_scaled = 'informer_loss_functions_results_scaled.csv'
csv_name_unscaled = 'informer_loss_functions_results_unscaled.csv'

# Convert the results into a DataFrame and save as CSV
informer_df_scaled = convert_results_into_df(informer_results_scaled, path_dir, csv_name_scaled)
informer_df_unscaled = convert_results_into_df(informer_results_unscaled, path_dir, csv_name_unscaled)
informer_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.5078  0.7126  0.4996  0.5640
              2         24        0.4878  0.6985  0.4790  0.5528
              1         96        0.8278  0.9098  0.6768  0.7216
              2         96        0.8712  0.9334  0.6843  0.7403
              1         168       0.9250  0.9618  0.7023  0.7619
              2         168       0.8726  0.9341  0.6942  0.7400
RMSE          1         24        0.5071  0.7121  0.4983  0.5636
              2         24        0.4826  0.6947  0.4776  0.5498
              1         96        0.8019  0.8955  0.6663  0.7102
              2         96        0.8538  0.9240  0.6723  0.7329
              1         168       0.8935  0.9452  0.6883  0.7488
              2         168       0.9231  0.9608  0.6903  0.7611
MAE           1         24        0.4949  0.7035  0.4628  0.5568
              2         24        0.4903  0.7002  0.4557  0.5542
              1         96        0.9520  0.9757  0.6667  0.7739
              2         96        0.8675  0.9314  0.6356  0.7387
              1         168       0.9182  0.9582  0.6792  0.7591
              2         168       0.9472  0.9732  0.6660  0.7710

In [ ]:
informer_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                          
MSE           1         24        20262616.0  4501.4014  3016.6672  0.2238
              2         24        19436216.0  4408.6523  2875.5586  0.2192
              1         96        35706256.0  5975.4712  4163.1543  0.2976
              2         96        38053568.0  6168.7573  4203.2881  0.3072
              1         168       41039112.0  6406.1777  4328.8550  0.3192
              2         168       37693112.0  6139.4717  4274.0054  0.3059
RMSE          1         24        20156686.0  4489.6196  2999.1426  0.2232
              2         24        19259186.0  4388.5288  2873.7637  0.2182
              1         96        34543984.0  5877.4131  4100.0752  0.2927
              2         96        36986492.0  6081.6519  4114.8584  0.3029
              1         168       38815100.0  6230.1768  4220.9731  0.3104
              2         168       40210260.0  6341.1562  4216.4253  0.3159
MAE           1         24        19648216.0  4432.6309  2762.4336  0.2204
              2         24        19207616.0  4382.6494  2741.9597  0.2179
              1         96        41247308.0  6422.4067  4063.6636  0.3198
              2         96        37667352.0  6137.3735  3862.1619  0.3056
              1         168       39692484.0  6300.1973  4161.8623  0.3139
              2         168       40591140.0  6371.1177  4053.1790  0.3174

In [ ]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'informer_loss_functions_results_scaled.csv'
#csv_name_unscaled = 'informer_loss_functions_results_unscaled.csv'

# Average the iterations
informer_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
informer_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

inf_res_scaled = informer_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
inf_res_unscaled = informer_unscaled.groupby(['Pred_len', 'Loss_function']).mean().sort_index().drop('Iteration', axis=1)
inf_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.4926  0.7018  0.4592  0.5555
         MSE            0.4978  0.7055  0.4893  0.5584
         RMSE           0.4949  0.7034  0.4879  0.5567
96       MAE            0.9098  0.9536  0.6512  0.7563
         MSE            0.8495  0.9216  0.6805  0.7309
         RMSE           0.8278  0.9097  0.6693  0.7215
168      MAE            0.9327  0.9657  0.6726  0.7650
         MSE            0.8988  0.9479  0.6982  0.7509
         RMSE           0.9083  0.9530  0.6893  0.7550

In [ ]:
inf_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                          
24       MAE            19427916.0  4407.6401  2752.1967  0.2192
         MSE            19849416.0  4455.0269  2946.1129  0.2215
         RMSE           19707936.0  4439.0742  2936.4531  0.2207
96       MAE            39457330.0  6279.8901  3962.9127  0.3127
         MSE            36879912.0  6072.1143  4183.2212  0.3024
         RMSE           35765238.0  5979.5325  4107.4668  0.2978
168      MAE            40141812.0  6335.6575  4107.5206  0.3157
         MSE            39366112.0  6272.8247  4301.4302  0.3125
         RMSE           39512680.0  6285.6665  4218.6992  0.3132

# 2. Standard Scaler PatchTST

In [ ]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "512"
lr = "0.0001"
model = "PatchTST"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# New log file path
log_file_path = f"{log_dir}/{model}_{country}.log"

patchtst_results_scaled, patchtst_results_unscaled = [], []

with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to notebook

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Command arguments
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 5 \
              --dec_in 5 \
              --c_out 5 \
              --des 'Exp' \
              --train_epochs 10 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --scaler_type standard \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()

            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics_scaled = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics_scaled, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(patchtst_results_scaled, scaled_metrics), (patchtst_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_512_24_loss_choice_for_DE', model='PatchTST', data='custom', root_path='/vol/cs-hu/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, seq_len=512, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=10, batch_size=32, patience=3, learning_rate=0.0001, des='Exp', lradj='type3', pct_start=0.3, 

In [ ]:
path_dir = './dataset_results'
csv_name_scaled = 'patchtst_loss_functions_results_scaled.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled.csv'


# Convert the results into a DataFrame and save as CSV
patchtst_df_scaled = convert_results_into_df(patchtst_results_scaled, path_dir, csv_name_scaled)
patchtst_df_unscaled = convert_results_into_df(patchtst_results_unscaled, path_dir, csv_name_unscaled)
patchtst_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.4551  0.6746  0.4416  0.5339
              2         24        0.4664  0.6829  0.4506  0.5405
              1         96        0.7439  0.8625  0.6113  0.6841
              2         96        0.7726  0.8790  0.6245  0.6971
              1         168       0.8226  0.9070  0.6497  0.7185
              2         168       0.8047  0.8970  0.6460  0.7106
RMSE          1         24        0.4513  0.6718  0.4435  0.5317
              2         24        0.4518  0.6722  0.4493  0.5320
              1         96        0.7535  0.8680  0.6172  0.6884
              2         96        0.7755  0.8806  0.6255  0.6985
              1         168       0.8276  0.9097  0.6519  0.7207
              2         168       0.8039  0.8966  0.6451  0.7103
MAE           1         24        0.4612  0.6791  0.4230  0.5375
              2         24        0.4455  0.6674  0.4187  0.5282
              1         96        0.7656  0.8750  0.5963  0.6940
              2         96        0.8202  0.9057  0.6112  0.7183
              1         168       0.8244  0.9080  0.6256  0.7193
              2         168       0.8161  0.9034  0.6275  0.7156

In [ ]:
patchtst_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                          
MSE           1         24        17261680.0  4154.7178  2581.1782  0.2066
              2         24        17762658.0  4214.5767  2639.2039  0.2096
              1         96        30875704.0  5556.5908  3633.8970  0.2767
              2         96        32792920.0  5726.5103  3746.3784  0.2852
              1         168       35247892.0  5936.9937  3905.6101  0.2958
              2         168       34226872.0  5850.3735  3899.1096  0.2915
RMSE          1         24        17305828.0  4160.0273  2609.1721  0.2068
              2         24        17345650.0  4164.8110  2657.5583  0.2071
              1         96        31262294.0  5591.2695  3673.3840  0.2784
              2         96        33044252.0  5748.4131  3757.2954  0.2863
              1         168       35287100.0  5940.2944  3912.6470  0.2960
              2         168       34170808.0  5845.5801  3891.9578  0.2913
MAE           1         24        17183404.0  4145.2871  2456.1895  0.2061
              2         24        16678795.0  4083.9680  2431.3040  0.2031
              1         96        31555720.0  5617.4478  3527.8032  0.2798
              2         96        34537172.0  5876.8335  3631.9407  0.2927
              1         168       35018344.0  5917.6299  3745.1887  0.2948
              2         168       33935912.0  5825.4536  3740.7778  0.2903

In [1]:
(5917.6299 + 5825.4536) / 2

5871.54175

In [ ]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'patchtst_loss_functions_results_scaled.csv'
#csv_name_unscaled = 'patchtst_loss_functions_results_unscaled.csv'

# Average the iterations
ptst_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
ptst_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

ptst_res_scaled = ptst_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.4534  0.6733  0.4208  0.5329
         MSE            0.4608  0.6788  0.4461  0.5372
         RMSE           0.4515  0.6720  0.4464  0.5318
96       MAE            0.7929  0.8903  0.6038  0.7061
         MSE            0.7583  0.8708  0.6179  0.6906
         RMSE           0.7645  0.8743  0.6214  0.6935
168      MAE            0.8203  0.9057  0.6265  0.7175
         MSE            0.8137  0.9020  0.6478  0.7146
         RMSE           0.8158  0.9032  0.6485  0.7155

In [ ]:
ptst_res_unscaled = ptst_unscaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                          
24       MAE            16931099.5  4114.6276  2443.7467  0.2046
         MSE            17512169.0  4184.6472  2610.1910  0.2081
         RMSE           17325739.0  4162.4192  2633.3652  0.2070
96       MAE            33046446.0  5747.1406  3579.8719  0.2862
         MSE            31834312.0  5641.5505  3690.1377  0.2810
         RMSE           32153273.0  5669.8413  3715.3397  0.2824
168      MAE            34477128.0  5871.5417  3742.9833  0.2925
         MSE            34737382.0  5893.6836  3902.3599  0.2937
         RMSE           34728954.0  5892.9373  3902.3024  0.2936

In [ ]:
shutil.rmtree("results_loss_scaled") # we do not need this directory and results anymore. If you need - comment this line

# Rename folder
os.rename("results_loss_unscaled", 'standard_unscaled')

# 3. MinMax Scaler (0, 1) Informer

We can use now "ReLU" activation function due to MinMax Scaler.

With BS 1036, ReLU - results are bad. (as twice as bad as with 32!)

In [3]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

# Arguments that will be used also for file names
model = "Informer"
dataset = 'DE_data.csv'
losses = ["MSE", "RMSE", "MAE"]
country = dataset[:2]

log_dir = f"logs/loss_choice/min_max_0_1_relu"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

log_file_path = f"{log_dir}/{model}_{country}.log"

In [4]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "96"
lr = "0.0001"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Lists to store the results
informer_results_scaled, informer_results_unscaled = [], []

# Log file
with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to .ipynb output cell

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Arguments for the command
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 5 \
              --dec_in 5 \
              --c_out 5 \
              --des 'Exp' \
              --train_epochs 10 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()
            
            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(informer_results_scaled, scaled_metrics), (informer_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_96_24_loss_choice_for_DE', model='Informer', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', seq_len=96, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=True, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=10, batch_size=32, patience=3, learning_rate=0.0

In [5]:
path_dir = './dataset_results'
csv_name_scaled = 'informer_loss_functions_results_scaled_minmax_0_1.csv'
csv_name_unscaled = 'informer_loss_functions_results_unscaled_minmax_0_1.csv'

# Convert the results into a DataFrame and save as CSV
informer_df_scaled = convert_results_into_df(informer_results_scaled, path_dir, csv_name_scaled)
informer_df_unscaled = convert_results_into_df(informer_results_unscaled, path_dir, csv_name_unscaled)
informer_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.0250  0.1580  0.1041  0.5580
              2         24        0.0229  0.1513  0.1014  0.5342
              1         96        0.0416  0.2040  0.1469  0.7224
              2         96        0.0419  0.2047  0.1414  0.7251
              1         168       0.0419  0.2048  0.1461  0.7255
              2         168       0.0428  0.2069  0.1482  0.7330
RMSE          1         24        0.0251  0.1586  0.1066  0.5599
              2         24        0.0230  0.1516  0.1000  0.5355
              1         96        0.0410  0.2026  0.1448  0.7173
              2         96        0.0419  0.2047  0.1425  0.7247
              1         168       0.0416  0.2039  0.1451  0.7222
              2         168       0.0432  0.2077  0.1486  0.7360
MAE           1         24        0.0243  0.1559  0.1003  0.5504
              2         24        0.0490  0.2214  0.1361  0.7819
              1         96        0.0454  0.2131  0.1421  0.7545
              2         96        0.0441  0.2099  0.1398  0.7433
              1         168       0.0439  0.2096  0.1448  0.7427
              2         168       0.0448  0.2116  0.1448  0.7496

In [6]:
informer_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                          
MSE           1         24        21478470.0  4634.4868  2927.1528  0.2304
              2         24        19517502.0  4417.8618  2843.4880  0.2197
              1         96        38027972.0  6166.6826  4192.3315  0.3071
              2         96        38741736.0  6224.2861  4020.8625  0.3100
              1         168       38667512.0  6218.3208  4159.8320  0.3098
              2         168       40122660.0  6334.2451  4268.5029  0.3156
RMSE          1         24        20226732.0  4497.4141  2958.5962  0.2236
              2         24        19474288.0  4412.9683  2820.4792  0.2194
              1         96        36910216.0  6075.3779  4103.1270  0.3026
              2         96        38137348.0  6175.5444  4037.7593  0.3075
              1         168       38455032.0  6201.2119  4132.5327  0.3090
              2         168       40330176.0  6350.6045  4271.2358  0.3164
MAE           1         24        21054224.0  4588.4883  2843.1777  0.2281
              2         24        43415168.0  6589.0186  3892.1006  0.3276
              1         96        40688736.0  6378.7725  4003.5076  0.3177
              2         96        38573712.0  6210.7739  3890.9275  0.3093
              1         168       39822152.0  6310.4795  4082.4695  0.3144
              2         168       41195184.0  6418.3477  4090.0542  0.3198

In [7]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'informer_loss_functions_results_scaled_minmax_0_1_relu.csv'
#csv_name_unscaled = 'informer_loss_functions_results_unscaled_minmax_0_1_relu.csv'

# Average the iterations
informer_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
informer_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

inf_res_scaled = informer_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
inf_res_unscaled = informer_unscaled.groupby(['Pred_len', 'Loss_function']).mean().sort_index().drop('Iteration', axis=1)
inf_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.0367  0.1886  0.1182  0.6662
         MSE            0.0239  0.1546  0.1027  0.5461
         RMSE           0.0241  0.1551  0.1033  0.5477
96       MAE            0.0447  0.2115  0.1410  0.7489
         MSE            0.0418  0.2044  0.1441  0.7237
         RMSE           0.0415  0.2036  0.1436  0.7210
168      MAE            0.0444  0.2106  0.1448  0.7461
         MSE            0.0424  0.2059  0.1472  0.7293
         RMSE           0.0424  0.2058  0.1468  0.7291

In [8]:
inf_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                          
24       MAE            32234696.0  5588.7534  3367.6392  0.2779
         MSE            20497986.0  4526.1743  2885.3204  0.2251
         RMSE           19850510.0  4455.1912  2889.5377  0.2215
96       MAE            39631224.0  6294.7732  3947.2175  0.3135
         MSE            38384854.0  6195.4844  4106.5970  0.3085
         RMSE           37523782.0  6125.4612  4070.4431  0.3051
168      MAE            40508668.0  6364.4136  4086.2618  0.3171
         MSE            39395086.0  6276.2830  4214.1675  0.3127
         RMSE           39392604.0  6275.9082  4201.8843  0.3127

# 4. MinMax Scaler (0, 1) PatchTST

In [ ]:
log_dir = f"logs/loss_choice/min_max_0_1_relu"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [4]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "512"
lr = "0.0001"
model = "PatchTST"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# New log file path
log_file_path = f"{log_dir}/{model}_{country}.log"

patchtst_results_scaled, patchtst_results_unscaled = [], []

with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to notebook

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Command arguments
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 5 \
              --dec_in 5 \
              --c_out 5 \
              --des 'Exp' \
              --train_epochs 10 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()

            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics_scaled = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics_scaled, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(patchtst_results_scaled, scaled_metrics), (patchtst_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_512_24_loss_choice_for_DE', model='PatchTST', data='custom', root_path='/vol/cs-hu/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, seq_len=512, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=10, batch_size=32, patience=3, learning_rate=0.0001, des='Exp', lradj='type3', pct_start=0.3, 

In [12]:
path_dir = './dataset_results'
csv_name_scaled = 'patchtst_loss_functions_results_scaled_minmax_0_1_relu.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_minmax_0_1_relu.csv'


# Convert the results into a DataFrame and save as CSV
patchtst_df_scaled = convert_results_into_df(patchtst_results_scaled, path_dir, csv_name_scaled)
patchtst_df_unscaled = convert_results_into_df(patchtst_results_unscaled, path_dir, csv_name_unscaled)
#patchtst_df_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
patchtst_df_scaled.round(4)

,Loss_function,Iteration,Pred_len,MSE,RMSE,MAE,RSE
0,MSE,1,24,0.0215,0.1465,0.0939,0.5172
1,MSE,2,24,0.0214,0.1464,0.0944,0.5169
2,MSE,1,96,0.0350,0.1872,0.1292,0.6629
3,MSE,2,96,0.0360,0.1898,0.1320,0.6722
4,MSE,1,168,0.0386,0.1965,0.1367,0.6961
5,MSE,2,168,0.0376,0.1939,0.1369,0.6870
6,RMSE,1,24,0.0215,0.1467,0.0941,0.5180
7,RMSE,2,24,0.0214,0.1463,0.0941,0.5166
8,RMSE,1,96,0.0351,0.1873,0.1296,0.6634
9,RMSE,2,96,0.0361,0.1901,0.1320,0.6731


In [6]:
patchtst_results_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
patchtst_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                          
MSE           1         24        17161886.0  4142.6904  2553.8210  0.2060
              2         24        17119326.0  4137.5508  2579.4404  0.2057
              1         96        30968164.0  5564.9048  3569.1465  0.2771
              2         96        32859850.0  5732.3511  3690.5352  0.2855
              1         168       35078208.0  5922.6860  3812.8645  0.2951
              2         168       34050108.0  5835.2471  3839.4783  0.2907
RMSE          1         24        17274290.0  4156.2349  2560.8831  0.2067
              2         24        17063130.0  4130.7544  2569.0222  0.2054
              1         96        31071448.0  5574.1768  3584.5562  0.2776
              2         96        32765476.0  5724.1138  3690.9900  0.2851
              1         168       34756880.0  5895.4966  3835.1350  0.2937
              2         168       33994724.0  5830.4995  3832.4016  0.2905
MAE           1         24        16924342.0  4113.9204  2418.1907  0.2046
              2         24        16638304.0  4079.0078  2411.2668  0.2028
              1         96        31024660.0  5569.9785  3461.2515  0.2774
              2         96        33443530.0  5783.0381  3538.7659  0.2880
              1         168       34850600.0  5903.4395  3733.3528  0.2941
              2         168       35550684.0  5962.4395  3720.8591  0.2971

In [6]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'patchtst_loss_functions_results_scaled_minmax_0_1_relu.csv'
#csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_minmax_0_1_relu.csv'

# Average the iterations
ptst_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
ptst_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

ptst_res_scaled = ptst_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.0216  0.1469  0.0899  0.5187
         MSE            0.0214  0.1464  0.0941  0.5171
         RMSE           0.0215  0.1465  0.0941  0.5173
96       MAE            0.0366  0.1913  0.1271  0.6775
         MSE            0.0355  0.1885  0.1306  0.6676
         RMSE           0.0356  0.1887  0.1308  0.6682
168      MAE            0.0391  0.1978  0.1342  0.7008
         MSE            0.0381  0.1952  0.1368  0.6916
         RMSE           0.0381  0.1951  0.1370  0.6911

In [7]:
ptst_res_unscaled = ptst_unscaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                          
24       MAE            16781323.0  4096.4641  2414.7288  0.2037
         MSE            17140606.0  4140.1206  2566.6307  0.2059
         RMSE           17168710.0  4143.4946  2564.9526  0.2060
96       MAE            32234095.0  5676.5083  3500.0087  0.2827
         MSE            31914007.0  5648.6279  3629.8408  0.2813
         RMSE           31918462.0  5649.1453  3637.7731  0.2813
168      MAE            35200642.0  5932.9395  3727.1060  0.2956
         MSE            34564158.0  5878.9666  3826.1714  0.2929
         RMSE           34375802.0  5862.9980  3833.7683  0.2921

In [15]:
# Rename folders
new_path_name = 'minmax_0_1_relu_unscaled'
shutil.rmtree("results_loss_scaled") # we do not need this directory and results anymore. If you need - comment this line
os.rename("results_loss_unscaled", new_path_name)

# 5. MinMax Scaler (0, 5) Informer


In [9]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

# Arguments that will be used also for file names
model = "Informer"
dataset = 'DE_data.csv'
losses = ["MSE", "RMSE", "MAE"]
country = dataset[:2]

log_dir = f"logs/loss_choice/min_max_0_5_relu"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

log_file_path = f"{log_dir}/{model}_{country}.log"

In [10]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "96"
lr = "0.0001"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Lists to store the results
informer_results_scaled, informer_results_unscaled = [], []

# Log file
with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to .ipynb output cell

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Arguments for the command
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 5 \
              --dec_in 5 \
              --c_out 5 \
              --des 'Exp' \
              --train_epochs 10 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --loss_fnc "{loss}" \
              --scaler_type minmax2 \
              --if_relu \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()
            
            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(informer_results_scaled, scaled_metrics), (informer_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_96_24_loss_choice_for_DE', model='Informer', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax2', seq_len=96, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=True, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=10, batch_size=32, patience=3, learning_rate=0.

	iters: 100, epoch: 1 | loss: 1.2955509
	speed: 0.0741s/iter; left time: 663.7305s
	iters: 200, epoch: 1 | loss: 0.9552944
	speed: 0.0421s/iter; left time: 373.4564s
	iters: 300, epoch: 1 | loss: 0.7724751
	speed: 0.0422s/iter; left time: 370.0596s
	iters: 400, epoch: 1 | loss: 0.6487309
	speed: 0.0443s/iter; left time: 383.8937s
	iters: 500, epoch: 1 | loss: 0.5562119
	speed: 0.0465s/iter; left time: 397.7432s
	iters: 600, epoch: 1 | loss: 0.4853845
	speed: 0.0464s/iter; left time: 392.8360s
	iters: 700, epoch: 1 | loss: 0.7075042
	speed: 0.0463s/iter; left time: 387.1326s
	iters: 800, epoch: 1 | loss: 0.5475918
	speed: 0.0459s/iter; left time: 378.8497s
	iters: 900, epoch: 1 | loss: 0.4198295
	speed: 0.0462s/iter; left time: 377.1901s
-------------------------------------------------------------------------------------
Epoch: 1
Cost time: 00h:00m:41.50s
Steps: 906 | Train Loss: 0.8569615 Vali Loss: 0.6873081 Test Loss: 0.7878568
Validation loss decreased (inf --> 0.687308).  Saving m

In [11]:
path_dir = './dataset_results'
csv_name_scaled = 'informer_loss_functions_results_scaled_minmax_0_5.csv'
csv_name_unscaled = 'informer_loss_functions_results_unscaled_minmax_0_5.csv'

# Convert the results into a DataFrame and save as CSV
informer_df_scaled = convert_results_into_df(informer_results_scaled, path_dir, csv_name_scaled)
informer_df_unscaled = convert_results_into_df(informer_results_unscaled, path_dir, csv_name_unscaled)
informer_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.5821  0.7630  0.5073  0.5389
              2         24        0.5617  0.7495  0.5070  0.5294
              1         96        0.9754  0.9876  0.7069  0.6995
              2         96        1.0202  1.0101  0.7201  0.7154
              1         168       1.0452  1.0224  0.7262  0.7244
              2         168       1.0489  1.0241  0.7294  0.7257
RMSE          1         24        0.5837  0.7640  0.5080  0.5396
              2         24        0.5531  0.7437  0.5012  0.5253
              1         96        0.9790  0.9894  0.7047  0.7008
              2         96        1.0342  1.0170  0.7238  0.7202
              1         168       1.0389  1.0192  0.7245  0.7222
              2         168       1.1415  1.0684  0.7632  0.7570
MAE           1         24        0.6487  0.8054  0.5156  0.5689
              2         24        0.5677  0.7535  0.4944  0.5322
              1         96        1.1340  1.0649  0.7311  0.7542
              2         96        1.0731  1.0359  0.7073  0.7337
              1         168       1.1463  1.0706  0.7319  0.7586
              2         168       1.0771  1.0378  0.7025  0.7354

In [12]:
informer_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                          
MSE           1         24        19813680.0  4451.2559  2843.8379  0.2213
              2         24        18792862.0  4335.0732  2853.9487  0.2155
              1         96        35925668.0  5993.8022  4037.1482  0.2985
              2         96        38556932.0  6209.4229  4125.3677  0.3092
              1         168       38648844.0  6216.8193  4132.9209  0.3098
              2         168       39350824.0  6273.0234  4192.4600  0.3126
RMSE          1         24        20211562.0  4495.7271  2858.9175  0.2235
              2         24        18355966.0  4284.3862  2809.9756  0.2130
              1         96        35732984.0  5977.7070  4005.1499  0.2977
              2         96        40138628.0  6335.5054  4191.2891  0.3155
              1         168       38251312.0  6184.7646  4112.1064  0.3082
              2         168       43871052.0  6623.5225  4414.6226  0.3300
MAE           1         24        22829454.0  4778.0176  2917.6660  0.2376
              2         24        19457170.0  4411.0283  2774.2051  0.2193
              1         96        42587312.0  6525.8955  4191.6289  0.3250
              2         96        40022552.0  6326.3379  4028.8872  0.3151
              1         168       43425884.0  6589.8320  4181.6504  0.3283
              2         168       38044424.0  6168.0161  3964.0671  0.3073

In [13]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'informer_loss_functions_results_scaled_minmax_0_5_relu.csv'
#csv_name_unscaled = 'informer_loss_functions_results_unscaled_minmax_0_5_relu.csv'

# Average the iterations
informer_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
informer_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

inf_res_scaled = informer_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
inf_res_unscaled = informer_unscaled.groupby(['Pred_len', 'Loss_function']).mean().sort_index().drop('Iteration', axis=1)
inf_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.6082  0.7795  0.5050  0.5505
         MSE            0.5719  0.7562  0.5071  0.5341
         RMSE           0.5684  0.7538  0.5046  0.5324
96       MAE            1.1036  1.0504  0.7192  0.7439
         MSE            0.9978  0.9988  0.7135  0.7074
         RMSE           1.0066  1.0032  0.7143  0.7105
168      MAE            1.1117  1.0542  0.7172  0.7470
         MSE            1.0471  1.0233  0.7278  0.7250
         RMSE           1.0902  1.0438  0.7438  0.7396

In [14]:
inf_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                          
24       MAE            21143312.0  4594.5229  2845.9355  0.2284
         MSE            19303271.0  4393.1646  2848.8933  0.2184
         RMSE           19283764.0  4390.0566  2834.4465  0.2183
96       MAE            41304932.0  6426.1167  4110.2581  0.3200
         MSE            37241300.0  6101.6125  4081.2579  0.3039
         RMSE           37935806.0  6156.6062  4098.2195  0.3066
168      MAE            40735154.0  6378.9241  4072.8588  0.3178
         MSE            38999834.0  6244.9214  4162.6904  0.3112
         RMSE           41061182.0  6404.1436  4263.3645  0.3191

# 6. MinMax Scaler (0, 5) PatchTST

In [11]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "512"
lr = "0.0001"
model = "PatchTST"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# New log file path
log_file_path = f"{log_dir}/{model}_{country}.log"

patchtst_results_scaled, patchtst_results_unscaled = [], []

with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to notebook

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Command arguments
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 5 \
              --dec_in 5 \
              --c_out 5 \
              --des 'Exp' \
              --train_epochs 10 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --loss_fnc "{loss}" \
              --scaler_type minmax2 \
              --if_relu \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()

            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics_scaled = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics_scaled, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(patchtst_results_scaled, scaled_metrics), (patchtst_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_512_24_loss_choice_for_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax2', seq_len=512, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=True, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=10, batch_size=32, patience=3, learning_rate=

In [12]:
path_dir = './dataset_results'
csv_name_scaled = 'patchtst_loss_functions_results_scaled_minmax_0_5_relu.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_minmax_0_5_relu.csv'


# Convert the results into a DataFrame and save as CSV
patchtst_df_scaled = convert_results_into_df(patchtst_results_scaled, path_dir, csv_name_scaled)
patchtst_df_unscaled = convert_results_into_df(patchtst_results_unscaled, path_dir, csv_name_unscaled)
patchtst_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.5360  0.7321  0.4716  0.5171
              2         24        0.5333  0.7303  0.4712  0.5158
              1         96        0.8908  0.9438  0.6565  0.6685
              2         96        0.9085  0.9531  0.6624  0.6750
              1         168       0.9642  0.9819  0.6851  0.6957
              2         168       0.9401  0.9696  0.6843  0.6870
RMSE          1         24        0.5414  0.7358  0.4735  0.5197
              2         24        0.5330  0.7301  0.4693  0.5156
              1         96        0.8934  0.9452  0.6558  0.6694
              2         96        0.8944  0.9457  0.6557  0.6698
              1         168       0.9804  0.9902  0.6853  0.7016
              2         168       0.9393  0.9691  0.6835  0.6867
MAE           1         24        0.5424  0.7365  0.4505  0.5202
              2         24        0.5394  0.7344  0.4493  0.5187
              1         96        0.8917  0.9443  0.6294  0.6688
              2         96        0.9266  0.9626  0.6355  0.6817
              1         168       0.9621  0.9809  0.6685  0.6950
              2         168       0.9993  0.9997  0.6711  0.7083

In [13]:
patchtst_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                          
MSE           1         24        17142486.0  4140.3486  2565.9302  0.2059
              2         24        17089198.0  4133.9082  2575.7898  0.2055
              1         96        31524144.0  5614.6367  3636.7886  0.2796
              2         96        33323196.0  5772.6245  3707.9365  0.2875
              1         168       34983416.0  5914.6782  3824.7483  0.2947
              2         168       34048708.0  5835.1270  3839.2070  0.2907
RMSE          1         24        17373884.0  4168.1992  2577.1997  0.2073
              2         24        17012618.0  4124.6357  2558.6548  0.2051
              1         96        31708800.0  5631.0566  3635.2244  0.2804
              2         96        32409162.0  5692.9043  3661.5996  0.2835
              1         168       35351660.0  5945.7261  3821.0962  0.2962
              2         168       33994148.0  5830.4502  3832.3152  0.2905
MAE           1         24        16739734.0  4091.4221  2419.3403  0.2034
              2         24        16871196.0  4107.4561  2425.1201  0.2042
              1         96        31304240.0  5595.0190  3466.7358  0.2786
              2         96        32648818.0  5713.9146  3491.8354  0.2846
              1         168       34872248.0  5905.2729  3729.8071  0.2942
              2         168       35697720.0  5974.7568  3709.1443  0.2977

In [14]:
# Uncomment the following lines if you want to read saved results
path_dir = './dataset_results'
csv_name_scaled = 'patchtst_loss_functions_results_scaled_minmax_0_5_relu.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_minmax_0_5_relu.csv'

# Average the iterations
ptst_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
ptst_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

ptst_res_scaled = ptst_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.5409  0.7354  0.4499  0.5194
         MSE            0.5346  0.7312  0.4714  0.5164
         RMSE           0.5372  0.7329  0.4714  0.5177
96       MAE            0.9091  0.9534  0.6325  0.6753
         MSE            0.8996  0.9485  0.6595  0.6718
         RMSE           0.8939  0.9455  0.6558  0.6696
168      MAE            0.9807  0.9903  0.6698  0.7016
         MSE            0.9521  0.9758  0.6847  0.6914
         RMSE           0.9598  0.9797  0.6844  0.6941

In [15]:
ptst_res_unscaled = ptst_unscaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                          
24       MAE            16805465.0  4099.4391  2422.2302  0.2038
         MSE            17115842.0  4137.1284  2570.8600  0.2057
         RMSE           17193251.0  4146.4175  2567.9272  0.2062
96       MAE            31976529.0  5654.4668  3479.2856  0.2816
         MSE            32423670.0  5693.6306  3672.3625  0.2835
         RMSE           32058981.0  5661.9805  3648.4120  0.2820
168      MAE            35284984.0  5940.0149  3719.4757  0.2960
         MSE            34516062.0  5874.9026  3831.9777  0.2927
         RMSE           34672904.0  5888.0881  3826.7057  0.2934

In [16]:
# Rename folders
new_path_name = 'minmax_0_5_relu_unscaled'
shutil.rmtree("results_loss_scaled") # we do not need this directory and results anymore. If you need - comment this line
os.rename("results_loss_unscaled", new_path_name)